# 基本使用
使用 TensorFlow, 你必须明白 TensorFlow:

* 使用图 (graph) 来表示计算任务.
* 在被称之为 会话 (Session) 的上下文 (context) 中执行图.
* 使用 tensor 表示数据.
* 通过 变量 (Variable) 维护状态.
* 使用 feed 和 fetch 可以为任意的操作(arbitrary operation) 赋值或者从其中获取数据.

# 综述
TensorFlow 是一个编程系统, 使用图来表示计算任务. 图中的节点被称之为 op (operation 的缩写). 一个 op 获得 0 个或多个 Tensor, 执行计算, 产生 0 个或多个 Tensor. 每个 Tensor 是一个类型化的多维数组. 例如, 你可以将一小组图像集表示为一个四维浮点数数组, 这四个维度分别是 [batch, height, width, channels].

一个 TensorFlow 图描述了计算的过程. 为了进行计算, 图必须在 会话 里被启动. 会话 将图的 op 分发到诸如 CPU 或 GPU 之类的 设备 上, 同时提供执行 op 的方法. 这些方法执行后, 将产生的 tensor 返回. 在 Python 语言中, 返回的 tensor 是 numpy ndarray 对象; 在 C 和 C++ 语言中, 返回的 tensor 是 tensorflow::Tensor 实例.

# 计算图

TensorFlow 程序通常被组织成一个构建阶段和一个执行阶段. 在构建阶段, op 的执行步骤 被描述成一个图. 在执行阶段, 使用会话执行执行图中的 op.

例如, 通常在构建阶段创建一个图来表示和训练神经网络, 然后在执行阶段反复执行图中的训练 op.

TensorFlow 支持 C, C++, Python 编程语言. 目前, TensorFlow 的 Python 库更加易用, 它提供了大量的辅助函数来简化构建图的工作, 这些函数尚未被 C 和 C++ 库支持.

三种语言的会话库 (session libraries) 是一致的.
# 构建图

构建图的第一步, 是创建源 op (source op). 源 op 不需要任何输入, 例如 常量 (Constant). 源 op 的输出被传递给其它 op 做运算.

Python 库中, op 构造器的返回值代表被构造出的 op 的输出, 这些返回值可以传递给其它 op 构造器作为输入.

TensorFlow Python 库有一个默认图 (default graph), op 构造器可以为其增加节点. 这个默认图对 许多程序来说已经足够用了. 阅读 Graph 类 文档 来了解如何管理多个图.

In [3]:
import tensorflow as tf
m1=tf.constant([[3,3]])
m2=tf.constant([[3],[3]])
output=tf.matmul(m1,m2)

# 在一个会话中启动图

构造阶段完成后, 才能启动图. 启动图的第一步是创建一个 Session 对象, 如果无任何创建参数, 会话构造器将启动默认图.

In [5]:
sess=tf.Session()
result=sess.run(output)
print(result)
sess.close()#要记得关闭，记得调用close

[[18]]


In [6]:
#也可以直接只用上下文表达式
with tf.Session() as sess:
    result=sess.run(output)
    print(result)

[[18]]


# 关于GPU和CPU的调用
在实现上, TensorFlow 将图形定义转换成分布式执行的操作, 以充分利用可用的计算资源(如 CPU 或 GPU). 一般你不需要显式指定使用 CPU 还是 GPU, TensorFlow 能自动检测. 如果检测到 GPU, TensorFlow 会尽可能地利用找到的第一个 GPU 来执行操作.

如果机器上有超过一个可用的 GPU, 除第一个外的其它 GPU 默认是不参与计算的. 为了让 TensorFlow 使用这些 GPU, 你必须将 op 明确指派给它们执行. with...Device 语句用来指派特定的 CPU 或 GPU 执行操作:

In [7]:
with tf.Session() as sess:
    with tf.device("/cpu:0"):
        matrix1 = tf.constant([[3., 3.]])
        matrix2 = tf.constant([[2.],[2.]])
        product = tf.matmul(matrix1, matrix2)
        result=sess.run(product)
        print(result)

[[ 12.]]


# 交互式使用
文档中的 Python 示例使用一个会话 Session 来 启动图, 并调用 Session.run() 方法执行操作.

为了便于使用诸如 IPython 之类的 Python 交互环境, 可以使用 InteractiveSession 代替 Session 类, 使用 Tensor.eval() 和 Operation.run() 方法代替 Session.run(). 这样可以避免使用一个变量来持有会话.

In [17]:
import tensorflow as tf
sess=tf.InteractiveSession()
x=tf.Variable([1,2])
a=tf.constant([4,5])
x.initializer.run()
sub=tf.subtract(x,a)#版本1.3.0改变了，原来的教程使用的tf.sub已经不能使用
print(sub.eval())

[-3 -3]


# 变量 Tensor
TensorFlow 程序使用 tensor 数据结构来代表所有的数据, 计算图中, 操作间传递的数据都是 tensor. 你可以把 TensorFlow tensor 看作是一个 n 维的数组或列表. 一个 tensor 包含一个静态类型 rank, 和 一个 shape.

In [27]:
import tensorflow as tf
state=tf.Variable(0,name='counter')
one=tf.constant(1)
newvalue=tf.add(state,one)
update=tf.assign(state,newvalue)
init_op=tf.global_variables_initializer()
with tf.Session() as tf:
    sess.run(init_op)
    print(sess.run(state))
    for i in range(10):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
4
5
6
7
8
9
10


# Fetch

为了取回操作的输出内容, 可以在使用 Session 对象的 run() 调用 执行图时, 传入一些 tensor, 这些 tensor 会帮助你取回结果. 在之前的例子里, 我们只取回了单个节点 state, 但是你也可以取回多个 tensor:

In [23]:
import tensorflow as tf
state=tf.Variable(0,name='counter')
one=tf.constant(1)
newvalue=tf.add(state,one)
update=tf.assign(state,newvalue)
init_op=tf.global_variables_initializer()
with tf.Session() as tf:
    sess.run(init_op)
    print(sess.run([state,one,update]))
    for i in range(10):
        sess.run(update)
        print(sess.run([state,one,update]))#fetch多个值

[1, 1, 1]
[3, 1, 3]
[5, 1, 5]
[7, 1, 7]
[9, 1, 9]
[11, 1, 11]
[13, 1, 13]
[15, 1, 15]
[17, 1, 17]
[19, 1, 19]
[21, 1, 21]


# Feed

上述示例在计算图中引入了 tensor, 以常量或变量的形式存储. TensorFlow 还提供了 feed 机制, 该机制 可以临时替代图中的任意操作中的 tensor 可以对图中任何操作提交补丁, 直接插入一个 tensor.

feed 使用一个 tensor 值临时替换一个操作的输出结果. 你可以提供 feed 数据作为 run() 调用的参数. feed 只在调用它的方法内有效, 方法结束, feed 就会消失. 最常见的用例是将某些特殊的操作指定为 "feed" 操作, 标记的方法是使用 tf.placeholder() 为这些操作创建占位符.

In [31]:
import tensorflow as tf
m1=tf.placeholder(tf.float32)
m2=tf.placeholder(tf.float32)
output=tf.matmul(m1,m2)
with tf.Session() as sess:
    print(sess.run(output,feed_dict={m1:1.2,m2:2.3}))

InvalidArgumentError: In[0] and In[1] ndims must be >= 2: 0
	 [[Node: MatMul_3 = BatchMatMul[T=DT_FLOAT, adj_x=false, adj_y=false, _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_Placeholder_1_0_0, _arg_Placeholder_2_0_1)]]

Caused by op 'MatMul_3', defined at:
  File "/home/wenshao/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wenshao/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wenshao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wenshao/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wenshao/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/wenshao/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177